In [1]:
import pandas as pd
from pandas.testing import assert_frame_equal
import numpy as np
import csv 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)

In [3]:
# Read csv
asthma_df = pd.read_csv('preprocessed_KNN_asthma.csv')

# Preprocessing

In [4]:
asthma_df.head(5)

,Unnamed: 0,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,bedtimeReport,waketimeReport,BASELINE_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Admission_LY,Age,AgeDiagnosis,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Family_his,Gender,Height,Hosp_treatme,Hospital_Trt,PetsHome,SchoolYear,SmokeHome,Sports,Vaccination,Weight,school_year_edit,Urbanisation,urbanisation2,totalSleepDuration,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay
0,1,1771258,0,Tue,school,3723.0,3723.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,137.0,238.4,355.4,605.0,1640.0,488.0,82.0,451.0,245.0,135.0,160.0,164.0,41.0,0.0,68.0,120.0,8.689362e+09,4.800952e+09,5.645238e+09,2.930000e+09,2.870000e+09,5.549524e+09,2.933333e+09,2.893333e+09,5.222381e+09,4.223381e+09,4.114286e+08,4.410196e+08,1.200000e+02,4.265675e+09,2.272018e+09,5.218571e+09,4.118000e+09,7.341429e+09,8.985714e+09,8.342857e+09,8.787500e+04,9.150000e+01,9.337500e+04,7.716667e+09,7.866667e+09,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,22,NaN,847826087.0,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33240.0,1266.0,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN
1,2,1771258,1,Wed,school,10015.0,10015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,0.0,82.0,168.0,1.326242e+09,7.360000e+01,7.528571e+09,7.075000e+01,9.200000e+01,7.600000e+01,7.033333e+09,8.233333e+09,7.800000e+01,9.900000e+01,1.130000e+02,1.298000e+02,9.516667e+09,9.166667e+09,3.764667e+09,1.085000e+02,8.525000e+01,9.340000e+01,9.588889e+09,1.018333e+09,1.423074e+09,1.078154e+09,1.042000e+02,9.183333e+09,9.300000e+01,180.0,22200.0,11760.0,0.0,84.0,67.0,121.0,21,8,847826087.0,10.0,12,8,11.3,100,13.1,0.2,2,0.9,2,1,3.0,3.47,32.18,37.49,25.72,29.69,10.83,3.09,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33960.0,2061.0,1.0,4355.0,94.0,102.0,76.0,Controlled asthma,NaN
2,3,1771258,2,Thu,school,3811.0,3811.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,561.0,159.0,356.0,26.0,591.0,109.0,114.0,456.0,155.0,727.0,46.0,139.0,210.0,6.0,93.0,0.0,71.0,122.0,9.030597e+09,7.916667e+09,7.316667e+09,8.033333e+09,7.800000e+01,7.614286e+09,7.466667e+09,7.814286e+09,9.850000e+01,1.034286e+09,1.005714e+09,1.096667e+09,1.006667e+09,1.050000e+02,1.090000e+02,8.650000e+01,8.866667e+09,1.028000e+02,7.820000e+01,8.520000e+01,9.400000e+01,1.000000e+02,8.925000e+01,9.650000e+01,8.200000e+01,300.0,19260.0,12360.0,1.0,76.0,64.0,129.0,22,7,847826087.0,6.0,9,4,9.9,62,13.3,8.2,89,0.0,0,0,1.0,6.80,37.86,48.29,26.94,33.27,16.22,4.94,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,31620.0,1384.0,1.0,727.0,90.0,96.0,77.0,Controlled asthma,NaN
3,4,1771258,3,Fri,school,4346.0,4346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,515.0,673.0,377.0,490.0,192.0,322.0,180.0,523.0,61.0,359.0,174.0,95.0,148.0,196.0,21.0,20.0,66.0,128.0,8.843151e+09,7.716667e+09,7.985714e+09,7.266667e+09,6.700000e+01,8.020000e+01,7.600000e+01,7.360000e+01,7.900000e+01,9.857143e+09,1.035000e+02,1.064286e+09,9.550000e+01,1.062857e+09,9.828571e+09,1.000000e+02,8.433333e+09,8.683333e+09,9.720000e+01,

# Delete colums 

In [5]:
# Just an index we dont need it
asthma_df = asthma_df.drop('Unnamed: 0', axis=1)
# As this is for data splitting for each child so we will leave the subject number in

In [14]:
asthma_df['CF_Asthma'].unique()

array(['Controlled asthma', 'Uncontrolled asthma', 'Cystic Fibrosis'],
      dtype=object)

In [7]:
asthma_df.head(5)

,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,bedtimeReport,waketimeReport,BASELINE_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Admission_LY,Age,AgeDiagnosis,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Family_his,Gender,Height,Hosp_treatme,Hospital_Trt,PetsHome,SchoolYear,SmokeHome,Sports,Vaccination,Weight,school_year_edit,Urbanisation,urbanisation2,totalSleepDuration,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay
0,1771258,0,Tue,school,3723.0,3723.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,137.0,238.4,355.4,605.0,1640.0,488.0,82.0,451.0,245.0,135.0,160.0,164.0,41.0,0.0,68.0,120.0,8.689362e+09,4.800952e+09,5.645238e+09,2.930000e+09,2.870000e+09,5.549524e+09,2.933333e+09,2.893333e+09,5.222381e+09,4.223381e+09,4.114286e+08,4.410196e+08,1.200000e+02,4.265675e+09,2.272018e+09,5.218571e+09,4.118000e+09,7.341429e+09,8.985714e+09,8.342857e+09,8.787500e+04,9.150000e+01,9.337500e+04,7.716667e+09,7.866667e+09,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,22,NaN,847826087.0,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33240.0,1266.0,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN
1,1771258,1,Wed,school,10015.0,10015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,0.0,82.0,168.0,1.326242e+09,7.360000e+01,7.528571e+09,7.075000e+01,9.200000e+01,7.600000e+01,7.033333e+09,8.233333e+09,7.800000e+01,9.900000e+01,1.130000e+02,1.298000e+02,9.516667e+09,9.166667e+09,3.764667e+09,1.085000e+02,8.525000e+01,9.340000e+01,9.588889e+09,1.018333e+09,1.423074e+09,1.078154e+09,1.042000e+02,9.183333e+09,9.300000e+01,180.0,22200.0,11760.0,0.0,84.0,67.0,121.0,21,8,847826087.0,10.0,12,8,11.3,100,13.1,0.2,2,0.9,2,1,3.0,3.47,32.18,37.49,25.72,29.69,10.83,3.09,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33960.0,2061.0,1.0,4355.0,94.0,102.0,76.0,Controlled asthma,NaN
2,1771258,2,Thu,school,3811.0,3811.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,561.0,159.0,356.0,26.0,591.0,109.0,114.0,456.0,155.0,727.0,46.0,139.0,210.0,6.0,93.0,0.0,71.0,122.0,9.030597e+09,7.916667e+09,7.316667e+09,8.033333e+09,7.800000e+01,7.614286e+09,7.466667e+09,7.814286e+09,9.850000e+01,1.034286e+09,1.005714e+09,1.096667e+09,1.006667e+09,1.050000e+02,1.090000e+02,8.650000e+01,8.866667e+09,1.028000e+02,7.820000e+01,8.520000e+01,9.400000e+01,1.000000e+02,8.925000e+01,9.650000e+01,8.200000e+01,300.0,19260.0,12360.0,1.0,76.0,64.0,129.0,22,7,847826087.0,6.0,9,4,9.9,62,13.3,8.2,89,0.0,0,0,1.0,6.80,37.86,48.29,26.94,33.27,16.22,4.94,1,15,7.0,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,No,Female,163.2,0.0,0.0,Yes,4.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,31620.0,1384.0,1.0,727.0,90.0,96.0,77.0,Controlled asthma,NaN
3,1771258,3,Fri,school,4346.0,4346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,515.0,673.0,377.0,490.0,192.0,322.0,180.0,523.0,61.0,359.0,174.0,95.0,148.0,196.0,21.0,20.0,66.0,128.0,8.843151e+09,7.716667e+09,7.985714e+09,7.266667e+09,6.700000e+01,8.020000e+01,7.600000e+01,7.360000e+01,7.900000e+01,9.857143e+09,1.035000e+02,1.064286e+09,9.550000e+01,1.062857e+09,9.828571e+09,1.000000e+02,8.433333e+09,8.683333e+09,9.720000e+01,7.400000e+01,1.0700

## everything that was NaN in that column, was unimportant and has to be far from 0 (I put 200)

In [13]:
EventDay_unique = asthma_df['EventDay'].unique()
EventDay_unique

array([200.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,
         3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,
        14.])

In [11]:
# 
asthma_df['EventDay'] = asthma_df['EventDay'].fillna(200)


In [12]:
# check if there are any NaNs in the dataframe
columns_with_nans = asthma_df.isnull().any(axis=0)
columns_with_nans_true = columns_with_nans[columns_with_nans].index
print(columns_with_nans_true)

Index(['HR05Perc', 'HR95Perc', 'HrAvgWake', 'awakeDuration', 'lightSleepDuration', 'deepSleepDuration', 'wakeUpCount', 'HRAvgSleep', 'HRMinSleep', 'HRMaxSleep', 'bedtimeReport', 'waketimeReport', 'BASELINE_PedsQL_score', 'pollenTotal', 'NO', 'NO2', 'Nox', 'Ox', 'AgeDiagnosis', 'Family_his', 'Sports', 'totalSleepDuration', 'AVGHR_daily', 'AVGHR_wake', 'AVGHR_sleep'], dtype='object')


In [31]:
zero_counts = asthma_df[columns_with_nans_true].isin([0]).sum()
print(zero_counts)
zero_counts[zero_counts == 0]

FG                                         0
FHX                                        0
FHN                                       32
TG                                         0
TN                                        16
TX                                         0
SQ                                       483
SP                                       483
DR                                      1049
RH                                       936
RHX                                      936
O3                                         0
Admission_LY                            1906
BirthMonth                                 0
BirthYear                                  0
BMI                                        0
CF_Asthma                                  0
DailyActiv                                 0
wear00                                   309
wear01                                   315
wear02                                   331
wear03                                   325
wear04    

FG             0
FHX            0
TG             0
TX             0
O3             0
BirthMonth     0
BirthYear      0
BMI            0
CF_Asthma      0
DailyActiv     0
AVGHR_daily    0
dtype: int64

# Data splitting to predict the day when a patient will have exacerbation
# Can't continue with NaNs for now. TBC.

In [46]:
# this is only a sample of imputation. We will use KNN imputated data instead.

#from sklearn.svm import OneClassSVM
#from sklearn.impute import SimpleImputer
#
## Prepare an imputer for handling missing values
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#
## Get unique SubjectNr
#unique_subjects = asthma_df['SubjectNr'].unique()
#
## Initialize a dictionary to store results for each subject
#anomaly_results = {}
#
#for subject in unique_subjects:
    ## Get data for all other subjects (not the current one)
    #other_subjects_data = asthma_df[asthma_df['SubjectNr'] != subject]
    ## Drop 'SubjectNr' column
    #other_subjects_data = other_subjects_data.drop(['SubjectNr','EventDay','DayNo'], axis=1)
    ## Handle missing values
    #other_subjects_data_imputed = imputer.fit_transform(other_subjects_data)
    ## Fit OneClassSVM on the imputed data
    #osvm = OneClassSVM(nu=0.1, kernel='sigmoid', gamma=0.01)
    #osvm.fit(other_subjects_data_imputed)
#
    ## predict anomalies on the current subject's data
    #subject_data = asthma_df[asthma_df['SubjectNr'] == subject]
    #subject_data = subject_data.drop(['SubjectNr','EventDay','DayNo'], axis=1)
    ## Handle missing values
    #subject_data_imputed = imputer.transform(subject_data)
#
    ## Get anomaly scores
    #anomaly_scores = osvm.decision_function(subject_data_imputed)
    ## Get binary labels for inliers (1) and outliers (-1)
    #anomaly_labels = osvm.predict(subject_data_imputed)
#
    ## Store results in the dictionary
    #anomaly_results[subject] = {
        #"scores": anomaly_scores,
        #"labels": anomaly_labels,
    #}
#

In [43]:
from sklearn.svm import OneClassSVM

groups = asthma_df['SubjectNr'].values  # use 'SubjectNr' to define the groups

# y = asthma_df['EventDay'].reset_index(drop=True)
X = asthma_df.drop(['EventDay','DayNo', 'SubjectNr'], axis=1).reset_index(drop=True)

logo = LeaveOneGroupOut()

# specify osvm parameters
osvm = OneClassSVM(nu=0.1, kernel='sigmoid', gamma=0.01)

for train_index, test_index in logo.split(X, None, groups):  # 'None' instead of 'y' 
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]

    # Fit the model to training data
    osvm.fit(X_train)

    # Predict the anomalous instances in the test set
    y_pred = osvm.predict(X_test)


ValueError: Input X contains NaN.
OneClassSVM does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values